# TM10007 Assignment template

In [ ]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [29]:
import pandas as pd

# Data loading functions. Uncomment the one you want to use
from worcliver.load_data import load_data


data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

#Data deviden in bengign and malign 
benign = data[data['label'] == 'benign']
malignant = data[data['label'] == 'malignant']

benign.to_csv('benign_data.csv', index=True)
malignant.to_csv('malignant_data.csv', index=True)


#statistics 
# Gemiddelden
benign_mean = benign.mean(numeric_only=True)
malignant_mean = malignant.mean(numeric_only=True)

# Standaarddeviaties
benign_std = benign.std(numeric_only=True)
malignant_std = malignant.std(numeric_only=True)

#Print statistiek 
benign_stats = pd.DataFrame({
    'mean': benign.mean(numeric_only=True),
    'std': benign.std(numeric_only=True)
})
print(benign_stats)
benign_stats.T.to_csv('benign_stats.csv')


malignant_stats = pd.DataFrame({
    'mean': malignant.mean(numeric_only=True),
    'std': malignant.std(numeric_only=True)
})
malignant_stats.T.to_csv('malignant_stats.csv')




The number of samples: 186
The number of columns: 494
                                                          mean          std
PREDICT_original_sf_compactness_avg_2.5D              0.805954     0.076541
PREDICT_original_sf_compactness_std_2.5D              0.068240     0.050810
PREDICT_original_sf_rad_dist_avg_2.5D                28.539695    11.786114
PREDICT_original_sf_rad_dist_std_2.5D                 4.498306     2.419078
PREDICT_original_sf_roughness_avg_2.5D                8.326638     3.338394
...                                                        ...          ...
PREDICT_original_phasef_phasesym_peak_position_...    0.000000     0.000000
PREDICT_original_phasef_phasesym_range_WL3_N5         0.434737     0.103427
PREDICT_original_phasef_phasesym_energy_WL3_N5      681.212476  1050.837965
PREDICT_original_phasef_phasesym_quartile_range...    0.088811     0.094786
PREDICT_original_phasef_phasesym_entropy_WL3_N5      11.990334     1.849521

[493 rows x 2 columns]
